In [2]:
import re

import pandas as pd
from rapidfuzz import fuzz

from copy import copy
from scipy import stats
import warnings
warnings.simplefilter('ignore')

import numpy as np
from statsmodels.api import add_constant, OLS
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf
import numpy as np


# testing weak-form efficiency

In [4]:
# Data processing
df = pd.read_csv('./stock_price.csv')
df = df [['TRADEDATE','CLOSE', 'TICKER']]
df2 = pd.read_excel('IMOEX.xlsx')

df2['TRADEDATE'] = pd.to_datetime(df2['TRADEDATE'],format='%d.%m.%Y' )
df= pd.concat([df, df2])
df['CLOSE'] = pd.to_numeric(df['CLOSE'], errors='coerce')

df = df.loc[df['TRADEDATE'] != 'TRADEDATE']
df = df.dropna()
df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])


df = df.sort_values(by='TRADEDATE', ascending=True)

df

,TRADEDATE,CLOSE,TICKER
17317,2021-02-01,35.2350,RUAL
49124,2021-02-01,88.4000,NKNC
45772,2021-02-01,70.3400,AFLT
59471,2021-02-01,0.1544,MRKU
20793,2021-02-01,464.0000,MDMG
...,...,...,...
30671,2024-05-17,1679.0000,TRNFP
29833,2024-05-17,746.5000,TATN
58631,2024-05-17,22.8700,LSNG
49122,2024-05-17,16336.0000,AKRN


### Weak-form efficiency 

In [5]:
# Calculate returns with price on stock
data = df.pivot(index='TRADEDATE', columns='TICKER', values='CLOSE')
print(data.isnull().sum())

data = data.fillna(method='ffill').fillna(method='bfill')
returns = data
returns

TICKER
AFKS      1
AFLT      1
AGRO      5
AKRN      3
ALRS      1
       ... 
VKCO    250
VSMO      3
VTBR      1
WUSH    458
YNDX      4
Length: 85, dtype: int64


TICKER,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,CARM,...,TRMK,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX
TRADEDATE,,,,,,,,,,,,,,,,,,,,,
2021-02-01,32.475,70.34,897.0,6062.0,100.61,249.0,466.20,1636.5,53.74,3.3180,...,59.30,143050.0,0.5710,2.844,139.00,897.4,21780.0,0.036945,185.40,4847.4
2021-02-02,33.316,70.14,902.8,6096.0,102.15,246.5,466.20,1660.0,54.37,3.3180,...,59.40,143350.0,0.5710,2.829,139.00,897.4,21820.0,0.036945,185.40,4997.0
2021-02-03,33.650,70.68,889.4,6102.0,103.66,249.0,466.20,1665.5,54.28,3.3180,...,58.24,144100.0,0.5710,2.848,139.00,897.4,21780.0,0.037000,185.40,4917.2
2021-02-04,33.963,70.56,894.0,6122.0,102.61,250.0,466.20,1655.0,55.08,3.3180,...,58.40,143900.0,0.5710,2.871,139.00,897.4,21980.0,0.037380,185.40,5016.6
2021-02-05,34.992,70.82,893.0,6090.0,102.32,251.0,466.20,1659.0,54.55,3.3180,...,58.74,144750.0,0.5710,2.861,139.00,897.4,22040.0,0.037885,185.40,4973.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-13,27.038,52.11,1545.4,16936.0,79.58,903.5,649.60,3401.0,332.10,2.3495,...,225.46,1643.0,0.9857,2.204,39.50,577.6,39520.0,0.023100,304.33,4407.0
2024-05-14,27.999,54.19,1499.0,16800.0,79.23,901.0,643.55,3395.0,332.80,2.3560,...,224.90,1671.5,0.9932,2.189,39.10,603.6,39460.0,0.023060,322.80,4489.8
2024-05-15,28.980,56.75,1517.8,16772.0,79.70,901.0,640.35,3378.0,339.73,2.4070,...,223.46,1667.5,1.0011,2.153,38.85,616.4,39440.0,0.023205,314.62,4355.2


In [6]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# Функция для выполнения теста ADF и возвращения результатов в виде Series
def adf_test(series):
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    return out

# Сбор всех результатов в один DataFrame
results_dict = {}

for company in returns.columns:
    results_dict[company] = adf_test(returns[company])

# Преобразование словаря результатов в DataFrame
results_df = pd.DataFrame(results_dict)

# Транспонирование DataFrame для желаемого формата (компании в столбцах)
results_df = results_df.T

# Добавление столбца с результатом гипотезы
results_df['null hypothesis'] = results_df['p-value'].apply(lambda p: 'reject' if p <= 0.05 else 'not reject')

# Вывод таблицы
print(results_df)

# Вывод интерпретации результатов (опционально, если требуется дополнительное пояснение)
for company in returns.columns:
    p_value = results_df.loc[company, 'p-value']
    if p_value <= 0.05:
        print(f"{company}: Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary.")
    else:
        print(f"{company}: Weak evidence against the null hypothesis, time series has a unit root, indicating it is non-stationary.")



      ADF Test Statistic   p-value  #Lags Used  Number of Observations Used  \
AFKS           -1.608600  0.479272         4.0                        815.0   
AFLT           -1.381023  0.591340         9.0                        810.0   
AGRO           -1.078828  0.723441         0.0                        819.0   
AKRN           -1.763257  0.398850        20.0                        799.0   
ALRS           -0.938766  0.774938        11.0                        808.0   
...                  ...       ...         ...                          ...   
VKCO           -1.507248  0.529923         7.0                        812.0   
VSMO           -1.925412  0.320203        10.0                        809.0   
VTBR           -1.103302  0.713753        15.0                        804.0   
WUSH            0.047429  0.962283        20.0                        799.0   
YNDX           -1.368300  0.597376         7.0                        812.0   

      Critical Value (1%)  Critical Value (5%)  Cri

In [6]:
results_df

,ADF Test Statistic,p-value,#Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%),null hypothesis
AFKS,-1.608600,0.479272,4.0,815.0,-3.438399,-2.865093,-2.568662,not reject
AFLT,-1.381023,0.591340,9.0,810.0,-3.438449,-2.865115,-2.568674,not reject
AGRO,-1.078828,0.723441,0.0,819.0,-3.438360,-2.865075,-2.568653,not reject
AKRN,-1.763257,0.398850,20.0,799.0,-3.438561,-2.865164,-2.568700,not reject
ALRS,-0.938766,0.774938,11.0,808.0,-3.438469,-2.865124,-2.568678,not reject
...,...,...,...,...,...,...,...,...
VKCO,-1.507248,0.529923,7.0,812.0,-3.438429,-2.865106,-2.568669,not reject
VSMO,-1.925412,0.320203,10.0,809.0,-3.438459,-2.865119,-2.568676,not reject
VTBR,-1.103302,0.713753,15.0,804.0,-3.438510,-2.865142,-2.568688,not reject
WUSH,0.047429,0.962283,20.0,799.0,-3.438561,-2.865164,-2.568700,not reject


show rejected companies

In [7]:
results_df.loc[results_df['null hypothesis']=='reject']

,ADF Test Statistic,p-value,#Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%),null hypothesis
DIAS,-4.029998,0.001261,17.0,802.0,-3.438530,-2.865151,-2.568693,reject
HNFG,-3.157041,0.022610,21.0,798.0,-3.438571,-2.865169,-2.568702,reject
HYDR,-3.154677,0.022763,1.0,818.0,-3.438369,-2.865080,-2.568655,reject


### Calculate returns

In [8]:
# Calculate returns
data = df.pivot(index='TRADEDATE', columns='TICKER', values='CLOSE')
print(data.isnull().sum())

data = data.fillna(method='ffill').fillna(method='bfill')
returns = np.log(data / data.shift(1)).dropna()
returns

TICKER
AFKS      1
AFLT      1
AGRO      5
AKRN      3
ALRS      1
       ... 
VKCO    250
VSMO      3
VTBR      1
WUSH    458
YNDX      4
Length: 85, dtype: int64


TICKER,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,CARM,...,TRMK,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX
TRADEDATE,,,,,,,,,,,,,,,,,,,,,
2021-02-02,0.025567,-0.002847,0.006445,0.005593,0.015191,-0.010091,0.000000,0.014258,0.011655,0.000000,...,0.001685,0.002095,0.000000,-0.005288,0.000000,0.000000,0.001835,0.000000,0.000000,0.030395
2021-02-03,0.009975,0.007669,-0.014954,0.000984,0.014674,0.010091,0.000000,0.003308,-0.001657,0.000000,...,-0.019722,0.005218,0.000000,0.006694,0.000000,0.000000,-0.001835,0.001488,0.000000,-0.016098
2021-02-04,0.009259,-0.001699,0.005159,0.003272,-0.010181,0.004008,0.000000,-0.006324,0.014631,0.000000,...,0.002743,-0.001389,0.000000,0.008043,0.000000,0.000000,0.009141,0.010218,0.000000,0.020013
2021-02-05,0.029848,0.003678,-0.001119,-0.005241,-0.002830,0.003992,0.000000,0.002414,-0.009669,0.000000,...,0.005805,0.005890,0.000000,-0.003489,0.000000,0.000000,0.002726,0.013419,0.000000,-0.008608
2021-02-08,0.016467,0.003665,0.002014,-0.000986,0.006138,0.000000,0.000000,0.015551,0.015823,0.000000,...,0.007463,0.012017,0.000000,0.011123,0.000000,0.000000,0.023319,0.010503,0.000000,0.017302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-13,0.004374,0.024477,-0.020242,0.006279,0.015322,-0.002211,0.042774,0.006342,0.014742,0.007476,...,0.006943,0.004270,-0.022371,-0.007684,0.000000,0.004859,0.002534,0.000866,-0.000821,0.011594
2024-05-14,0.034926,0.039140,-0.030485,-0.008063,-0.004408,-0.002771,-0.009357,-0.001766,0.002106,0.002763,...,-0.002487,0.017198,0.007580,-0.006829,-0.010178,0.044030,-0.001519,-0.001733,0.058920,0.018614
2024-05-15,0.034437,0.046159,0.012464,-0.001668,0.005915,0.000000,-0.004985,-0.005020,0.020609,0.021416,...,-0.006423,-0.002396,0.007923,-0.016583,-0.006414,0.020984,-0.000507,0.006268,-0.025667,-0.030438


In [9]:
# List of eliminated companies 
list_returns = returns.columns
tickers = df['TICKER'].unique()

diff_list2 = [item for item in tickers if item not in list_returns]
print("List of eliminated companies:", diff_list2)

List of eliminated companies: []


### Check series stationarity

In [10]:
import pandas as pd
from statsmodels.tsa.stattools import adfuller

# Функция для выполнения теста ADF и возвращения результатов в виде Series
def adf_test(series):
    result = adfuller(series.dropna(), autolag='AIC')
    labels = ['ADF Test Statistic', 'p-value', '#Lags Used', 'Number of Observations Used']
    out = pd.Series(result[0:4], index=labels)
    for key, value in result[4].items():
        out[f'Critical Value ({key})'] = value
    return out

# Сбор всех результатов в один DataFrame
results_dict = {}

for company in returns.columns:
    results_dict[company] = adf_test(returns[company])

# Преобразование словаря результатов в DataFrame
results_df = pd.DataFrame(results_dict)

# Транспонирование DataFrame для желаемого формата (компании в столбцах)
results_df = results_df.T

# Добавление столбца с результатом гипотезы
results_df['null hypothesis'] = results_df['p-value'].apply(lambda p: 'reject' if p <= 0.05 else 'accept')

# Вывод таблицы
print(results_df)

# Вывод интерпретации результатов (опционально, если требуется дополнительное пояснение)
for company in returns.columns:
    p_value = results_df.loc[company, 'p-value']
    if p_value <= 0.05:
        print(f"{company}: Strong evidence against the null hypothesis, reject the null hypothesis. Data has no unit root and is stationary.")
    else:
        print(f"{company}: Weak evidence against the null hypothesis, time series has a unit root, indicating it is non-stationary.")



      ADF Test Statistic       p-value  #Lags Used  \
AFKS          -12.282972  8.144464e-23         3.0   
AFLT          -11.311542  1.235508e-20         8.0   
AGRO          -12.898053  4.306560e-24         5.0   
AKRN           -7.002260  7.270897e-10        18.0   
ALRS          -10.734030  2.930106e-19        10.0   
...                  ...           ...         ...   
VKCO           -5.512473  1.958091e-06        21.0   
VSMO           -9.724414  9.357018e-17         9.0   
VTBR           -6.890375  1.360505e-09        16.0   
WUSH           -6.095278  1.013582e-07        19.0   
YNDX          -12.395665  4.681538e-23         6.0   

      Number of Observations Used  Critical Value (1%)  Critical Value (5%)  \
AFKS                        815.0            -3.438399            -2.865093   
AFLT                        810.0            -3.438449            -2.865115   
AGRO                        813.0            -3.438419            -2.865102   
AKRN                        800.0  

In [13]:
results_df

,ADF Test Statistic,p-value,#Lags Used,Number of Observations Used,Critical Value (1%),Critical Value (5%),Critical Value (10%),null hypothesis
AFKS,-12.282972,8.144464e-23,3.0,815.0,-3.438399,-2.865093,-2.568662,reject
AFLT,-11.311542,1.235508e-20,8.0,810.0,-3.438449,-2.865115,-2.568674,reject
AGRO,-12.898053,4.306560e-24,5.0,813.0,-3.438419,-2.865102,-2.568667,reject
AKRN,-7.002260,7.270897e-10,18.0,800.0,-3.438551,-2.865160,-2.568697,reject
ALRS,-10.734030,2.930106e-19,10.0,808.0,-3.438469,-2.865124,-2.568678,reject
...,...,...,...,...,...,...,...,...
VKCO,-5.512473,1.958091e-06,21.0,797.0,-3.438581,-2.865173,-2.568705,reject
VSMO,-9.724414,9.357018e-17,9.0,809.0,-3.438459,-2.865119,-2.568676,reject
VTBR,-6.890375,1.360505e-09,16.0,802.0,-3.438530,-2.865151,-2.568693,reject
WUSH,-6.095278,1.013582e-07,19.0,799.0,-3.438561,-2.865164,-2.568700,reject


CORRORELOGRAMS ADF


In [28]:
# ADF viz
# for company in returns.columns:
#     plot_acf(returns[company].dropna(), title=f'Autocorrelation for {company} Returns', lags=20)
#     plt.show()

CAPM 22 based on 21

In [12]:
# Choose data before 23 feb 2022
returns = returns.reset_index()
df_feb23 = returns.loc[returns['TRADEDATE'] < '2022-02-09']
# Convert datetime column to numerical value
df_feb23['TRADEDATE'] = pd.to_datetime(df_feb23['TRADEDATE'])
df_feb23

TICKER,TRADEDATE,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,...,TRMK,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX
0,2021-02-02,0.025567,-0.002847,0.006445,0.005593,0.015191,-0.010091,0.0,0.014258,0.011655,...,0.001685,0.002095,0.0,-0.005288,0.000000,0.000000,0.001835,0.000000,0.0,0.030395
1,2021-02-03,0.009975,0.007669,-0.014954,0.000984,0.014674,0.010091,0.0,0.003308,-0.001657,...,-0.019722,0.005218,0.0,0.006694,0.000000,0.000000,-0.001835,0.001488,0.0,-0.016098
2,2021-02-04,0.009259,-0.001699,0.005159,0.003272,-0.010181,0.004008,0.0,-0.006324,0.014631,...,0.002743,-0.001389,0.0,0.008043,0.000000,0.000000,0.009141,0.010218,0.0,0.020013
3,2021-02-05,0.029848,0.003678,-0.001119,-0.005241,-0.002830,0.003992,0.0,0.002414,-0.009669,...,0.005805,0.005890,0.0,-0.003489,0.000000,0.000000,0.002726,0.013419,0.0,-0.008608
4,2021-02-08,0.016467,0.003665,0.002014,-0.000986,0.006138,0.000000,0.0,0.015551,0.015823,...,0.007463,0.012017,0.0,0.011123,0.000000,0.000000,0.023319,0.010503,0.0,0.017302
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257,2022-02-02,-0.017931,0.001400,0.006419,0.006789,0.023211,-0.027815,0.0,0.010050,-0.001488,...,0.003706,-0.009779,0.0,-0.001229,-0.019480,-0.035080,-0.003364,-0.006933,0.0,-0.013245
258,2022-02-03,-0.028023,-0.026574,-0.036110,-0.059914,-0.021039,-0.043675,0.0,-0.018019,-0.020438,...,-0.009290,-0.002715,0.0,-0.011961,0.005474,-0.050213,0.003784,-0.027618,0.0,-0.043945
259,2022-02-04,0.004059,0.008581,0.001515,0.101916,0.009587,0.017700,0.0,0.001453,0.010334,...,0.025018,0.029133,0.0,0.000000,-0.002277,-0.035851,-0.002942,0.000121,0.0,-0.004160
260,2022-02-07,-0.017005,0.000000,0.014094,0.108461,-0.003099,-0.005277,0.0,0.012270,0.003005,...,0.061527,-0.009950,0.0,-0.003325,-0.029148,0.001720,-0.000421,-0.003643,0.0,-0.009870


In [14]:
# Calculate alpha and beta coeff
data_slope = []
for tickers in df_feb23.columns:
    if df_feb23[tickers].dtype != 'datetime64[ns]':
        beta, alpha = np.polyfit(df_feb23['IMOEX'], df_feb23[tickers],1)
        
        df_slope = {'TICKER': tickers, 'beta': round(beta, 5), 'alpha': round(alpha, 5)}
        data_slope.append(df_slope)
        print('Beta for {} stock is = {} and alpha is = {}'.format('AFLT', round(beta,5), round(alpha,5)))
        
df_coeff = pd.DataFrame(data_slope)
df_coeff


Beta for AFLT stock is = 0.89618 and alpha is = -0.00204
Beta for AFLT stock is = 0.63574 and alpha is = -0.00093
Beta for AFLT stock is = 0.64578 and alpha is = 0.00051
Beta for AFLT stock is = 0.5345 and alpha is = 0.00332
Beta for AFLT stock is = 0.78986 and alpha is = 0.00047
Beta for AFLT stock is = 0.60792 and alpha is = 0.00306
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.71502 and alpha is = -0.00067
Beta for AFLT stock is = 0.77763 and alpha is = 0.00143
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.72124 and alpha is = 0.00027
Beta for AFLT stock is = 0.64587 and alpha is = 0.00078
Beta for AFLT stock is = 0.76266 and alpha is = -0.00245
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.38485 and alpha is = -0.00131
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.84185 and alpha is = 0.00055
Beta for AFLT stock is = 0.5728 and alpha is = -0.00171
Beta for AFLT stock 

,TICKER,beta,alpha
0,AFKS,0.89618,-0.00204
1,AFLT,0.63574,-0.00093
2,AGRO,0.64578,0.00051
3,AKRN,0.53450,0.00332
4,ALRS,0.78986,0.00047
...,...,...,...
80,VKCO,0.52388,-0.00159
81,VSMO,0.19323,0.00305
82,VTBR,0.97528,0.00030
83,WUSH,0.00000,0.00000


In [15]:
# Choose data after sanctions introdaction
df_sanctions_22 = returns.loc[returns['TRADEDATE'] > '2022-02-08']
df_sanctions_22 = df_sanctions_22.loc[df_sanctions_22['TRADEDATE'] < '2023-01-01']
df_sanctions_22

TICKER,TRADEDATE,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,...,TRMK,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX
262,2022-02-09,0.025018,0.034717,0.015302,0.033028,0.029368,-0.002619,0.0,0.011488,-0.000491,...,0.000000,0.016952,0.0,0.005807,0.053245,0.040494,-0.006555,0.012597,0.000000,0.037583
263,2022-02-10,-0.012307,0.009415,0.020166,0.032475,0.003916,-0.008780,0.0,0.007930,0.043571,...,0.043345,0.004932,0.0,0.008237,0.009972,-0.044437,0.000411,-0.019849,0.000000,0.011099
264,2022-02-11,-0.046504,-0.024734,-0.010949,-0.019051,-0.040890,-0.017794,0.0,-0.025036,-0.014791,...,-0.036758,-0.043915,0.0,-0.011966,-0.052247,-0.037918,-0.014067,-0.033984,0.000000,-0.103814
265,2022-02-14,-0.007314,-0.017301,-0.010328,-0.031918,-0.020223,-0.013556,0.0,-0.017046,0.017605,...,-0.028108,0.021360,0.0,-0.019280,0.020693,0.018971,0.006645,-0.005323,0.000000,0.012056
266,2022-02-15,0.052880,0.036330,0.017277,0.041362,0.002335,0.000910,0.0,0.014225,-0.000937,...,0.016615,0.025502,0.0,0.007168,0.038004,0.035276,0.007423,0.038949,0.000000,0.129967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,2022-12-26,-0.002495,0.009860,-0.004169,-0.002980,0.001856,0.004165,0.0,-0.008106,0.008507,...,0.017300,-0.002911,0.0,0.003857,-0.009917,0.003541,-0.009121,0.010284,0.043350,-0.005396
466,2022-12-27,-0.001833,-0.005740,-0.004786,0.000442,-0.003378,-0.003331,0.0,0.023594,-0.003394,...,0.010386,0.001748,0.0,0.001538,-0.013378,0.001325,-0.012574,-0.006157,-0.014037,0.012399
467,2022-12-28,-0.007368,-0.005773,-0.009036,-0.000553,-0.015171,-0.000834,0.0,-0.002122,-0.006018,...,0.004908,0.000582,0.0,0.027295,-0.013560,-0.028196,0.007563,-0.031369,0.013007,-0.009973
468,2022-12-29,-0.000168,0.015593,0.010834,0.004413,0.013139,-0.000835,0.0,-0.001594,0.005018,...,0.006588,0.010992,0.0,0.004478,0.006803,0.002267,0.006675,0.008161,0.007194,0.002091


In [16]:
# Calculate expected return 
expected_returns = df_sanctions_22.copy()
for ticker in df_coeff['TICKER']:
    if ticker in df_sanctions_22.columns:
        df_coeff_ind = df_coeff.loc[df_coeff['TICKER']==ticker]
        expected_returns[ticker] = df_coeff_ind['alpha'].values[0] + df_coeff_ind['beta'].values[0] * df_sanctions_22['IMOEX']
        

print(expected_returns)

TICKER  TRADEDATE      AFKS      AFLT      AGRO      AKRN      ALRS      AQUA  \
262    2022-02-09  0.019717  0.014504  0.016188  0.016296  0.019646  0.017819   
263    2022-02-10  0.002095  0.002004  0.003490  0.005786  0.004115  0.005865   
264    2022-02-11 -0.029202 -0.020199 -0.019063 -0.012880 -0.023470 -0.015365   
265    2022-02-14 -0.018679 -0.012733 -0.011480 -0.006604 -0.014195 -0.008227   
266    2022-02-15  0.028059  0.020422  0.022199  0.021272  0.026998  0.023477   
..            ...       ...       ...       ...       ...       ...       ...   
465    2022-12-26  0.004346  0.003600  0.005112  0.007129  0.006099  0.007392   
466    2022-12-27  0.002852  0.002541  0.004035  0.006238  0.004782  0.006379   
467    2022-12-28 -0.006698 -0.004234 -0.002846  0.000542 -0.003635 -0.000100   
468    2022-12-29  0.001008  0.001232  0.002706  0.005138  0.003156  0.005128   
469    2022-12-30  0.000902  0.001157  0.002630  0.005075  0.003063  0.005056   

TICKER  ASTR      BANE     

In [17]:
# calulate abnormal returns

expected_returns22 = expected_returns

 
abnormal_returns22 = df_sanctions_22 - expected_returns22
abnormal_returns22 = abnormal_returns22.drop(columns='TRADEDATE')

abnormal_returns22 = pd.merge(abnormal_returns22, df_sanctions_22['TRADEDATE'], left_index=True, right_index=True)
abnormal_returns22

In [18]:
abnormal_returns22 = pd.merge(abnormal_returns22, df_sanctions_22['TRADEDATE'], left_index=True, right_index=True)
abnormal_returns22

,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,CARM,...,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX,TRADEDATE
262,0.005301,0.020213,-0.000886,0.016732,0.009722,-0.020438,0.0,-0.005200,-0.020799,0.0,...,0.011070,0.0,-0.000224,0.045777,0.029366,-0.014296,-0.011381,0.000000,0.020499,2022-02-09
263,-0.014403,0.007411,0.016676,0.026689,-0.000198,-0.014645,0.0,0.005300,0.038553,0.0,...,0.003725,0.0,0.007658,0.009371,-0.045265,-0.003531,-0.024650,0.000000,0.008881,2022-02-10
264,-0.017301,-0.004536,0.008114,-0.006170,-0.017420,-0.002429,0.0,-0.002695,0.007349,0.0,...,-0.036819,0.0,-0.002864,-0.040653,-0.020449,-0.011261,-0.004724,0.000000,-0.079630,2022-02-11
265,0.011364,-0.004568,0.001151,-0.025315,-0.006029,-0.005330,0.0,-0.003101,0.030613,0.0,...,0.025664,0.0,-0.013433,0.028187,0.030287,0.007182,0.012485,0.000000,0.027362,2022-02-14
266,0.024821,0.015909,-0.004922,0.020090,-0.024663,-0.022568,0.0,-0.009119,-0.028485,0.0,...,0.017407,0.0,-0.001442,0.027286,0.019272,-0.002117,0.005893,0.000000,0.105846,2022-02-15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,-0.006841,0.006260,-0.009281,-0.010109,-0.004242,-0.003227,0.0,-0.012531,0.001536,0.0,...,-0.004715,0.0,0.002581,-0.011396,0.001398,-0.013548,0.003034,0.043350,-0.009513,2022-12-26
466,-0.004686,-0.008281,-0.008821,-0.005796,-0.008160,-0.009709,0.0,0.020360,-0.009069,0.0,...,0.000340,0.0,0.000725,-0.014274,0.000055,-0.016678,-0.011782,-0.014037,0.009542,2022-12-27
467,-0.000670,-0.001539,-0.006190,-0.001095,-0.011536,-0.000735,0.0,0.002264,-0.003406,0.0,...,0.001708,0.0,0.029436,-0.010735,-0.023883,0.005517,-0.026600,0.013007,-0.004774,2022-12-28
468,-0.001176,0.014361,0.008127,-0.000725,0.009983,-0.005963,0.0,-0.003356,0.000943,0.0,...,0.010074,0.0,0.004235,0.006625,0.002075,0.002968,0.004544,0.007194,0.000789,2022-12-29


CAPM 2023

In [19]:
df_for_coeff = returns.loc[(returns['TRADEDATE'] >= '2022-01-01') & (returns['TRADEDATE'] <= '2022-12-31')]

data_slope = []
for tickers in df_for_coeff.columns:
    if df_for_coeff[tickers].dtype != 'datetime64[ns]':
        beta, alpha = np.polyfit(df_for_coeff['IMOEX'], df_for_coeff[tickers],1)
        
        df_slope = {'TICKER': tickers, 'beta': round(beta, 5), 'alpha': round(alpha, 5)}
        data_slope.append(df_slope)
        print('Beta for {} stock is = {} and alpha is = {}'.format('AFLT', round(beta,5), round(alpha,5)))
        
df_coeff = pd.DataFrame(data_slope)


df_sanctions_23 = returns.loc[(returns['TRADEDATE'] >= '2023-01-01') & (returns['TRADEDATE'] <= '2023-12-31')]

# Расчет ожидаемой доходности
expected_returns = df_sanctions_23.copy()
for ticker in df_coeff['TICKER']:
    if ticker in df_sanctions_23.columns:
        df_coeff_ind = df_coeff.loc[df_coeff['TICKER']==ticker]
        expected_returns[ticker] = df_coeff_ind['alpha'].values[0] + df_coeff_ind['beta'].values[0] * df_sanctions_23['IMOEX']

expected_returns23 = expected_returns
abnormal_returns23 = df_sanctions_23 - expected_returns23
abnormal_returns23 = abnormal_returns23.drop(columns='TRADEDATE')
abnormal_returns23 = pd.merge(abnormal_returns23, df_sanctions_23['TRADEDATE'], left_index=True, right_index=True)
abnormal_returns23


Beta for AFLT stock is = 0.83652 and alpha is = -0.00085
Beta for AFLT stock is = 0.7939 and alpha is = -0.00183
Beta for AFLT stock is = 0.58829 and alpha is = -0.00103
Beta for AFLT stock is = 0.36227 and alpha is = 0.00252
Beta for AFLT stock is = 0.69094 and alpha is = -0.00146
Beta for AFLT stock is = 0.61535 and alpha is = 0.00144
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.63102 and alpha is = -5e-05
Beta for AFLT stock is = 0.91025 and alpha is = 0.00306
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.41467 and alpha is = 0.0009
Beta for AFLT stock is = 0.66338 and alpha is = -0.00085
Beta for AFLT stock is = 0.67074 and alpha is = -0.00295
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.6201 and alpha is = -0.00098
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.76459 and alpha is = -0.00182
Beta for AFLT stock is = 1.08927 and alpha is = 1e-05
Beta for AFLT stock is

,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,CARM,...,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX,TRADEDATE
470,0.012922,-0.003356,0.011301,-0.000152,0.004503,0.002482,0.000000,0.020843,0.008341,0.000000,...,0.002199,0.000000,0.011793,0.041344,0.013321,-0.000753,-0.004126,0.024987,0.000523,2023-01-03
471,0.010187,0.019492,0.020496,-0.005091,0.001636,0.015464,0.000000,-0.003378,-0.002746,0.000000,...,0.000515,0.000000,-0.001862,0.004421,0.012268,-0.000290,0.003194,-0.004921,0.022404,2023-01-04
472,-0.007573,0.028261,0.014128,-0.002966,-0.001387,-0.003027,0.000000,-0.001732,0.001100,0.000000,...,0.003156,0.000000,0.001899,0.031532,0.004867,0.012771,0.001202,-0.000642,0.000782,2023-01-05
473,-0.004509,-0.003551,-0.006456,-0.013630,0.000530,-0.007972,0.000000,-0.003532,-0.001272,0.000000,...,0.001400,0.000000,0.006479,-0.001707,-0.008417,-0.007260,-0.004979,-0.004832,-0.001086,2023-01-06
474,-0.015283,0.023277,0.009550,-0.003379,0.004779,0.006435,0.000000,0.007887,-0.000673,0.000000,...,0.002781,0.000000,0.007827,0.034743,0.006628,0.010925,0.013947,-0.000024,0.006951,2023-01-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
719,-0.001015,-0.011285,0.025515,0.010840,-0.013025,0.009024,0.005600,0.026640,-0.003578,0.004120,...,0.000527,-0.007168,-0.004582,-0.007439,-0.006129,-0.003809,0.009195,0.008353,0.042331,2023-12-25
720,0.001949,0.000398,0.013669,-0.001486,0.007119,-0.015287,-0.017483,0.005502,-0.025982,-0.017464,...,-0.001670,0.016647,0.006603,-0.002449,-0.015708,-0.021161,0.001965,-0.002194,0.015746,2023-12-26
721,-0.000895,-0.009082,-0.007700,-0.002738,-0.006717,-0.015109,-0.013018,-0.015849,-0.021608,-0.008848,...,0.004196,-0.010749,-0.003301,0.001398,-0.000240,-0.005717,-0.011415,0.031472,0.049759,2023-12-27
722,-0.010192,-0.006182,-0.012183,-0.005702,-0.004566,0.007225,-0.014196,-0.024544,-0.012785,-0.002892,...,-0.001411,-0.006698,-0.011158,-0.026771,0.017774,-0.014418,-0.007545,-0.010561,-0.007399,2023-12-28


CAPM 2024

In [20]:
df_for_coeff = returns.loc[(returns['TRADEDATE'] >= '2023-01-01') & (returns['TRADEDATE'] <= '2023-12-31')]

data_slope = []
for tickers in df_for_coeff.columns:
    if df_for_coeff[tickers].dtype != 'datetime64[ns]':
        beta, alpha = np.polyfit(df_for_coeff['IMOEX'], df_for_coeff[tickers],1)
        
        df_slope = {'TICKER': tickers, 'beta': round(beta, 5), 'alpha': round(alpha, 5)}
        data_slope.append(df_slope)
        print('Beta for {} stock is = {} and alpha is = {}'.format('AFLT', round(beta,5), round(alpha,5)))
        
df_coeff = pd.DataFrame(data_slope)


df_sanctions_24 = returns.loc[(returns['TRADEDATE'] >= '2024-01-01')]

# Расчет ожидаемой доходности
expected_returns = df_sanctions_24.copy()
for ticker in df_coeff['TICKER']:
    if ticker in df_sanctions_24.columns:
        df_coeff_ind = df_coeff.loc[df_coeff['TICKER']==ticker]
        expected_returns[ticker] = df_coeff_ind['alpha'].values[0] + df_coeff_ind['beta'].values[0] * df_sanctions_24['IMOEX']

expected_returns24 = expected_returns
abnormal_returns24 = df_sanctions_24 - expected_returns24
abnormal_returns24 = abnormal_returns24.drop(columns='TRADEDATE')
abnormal_returns24 = pd.merge(abnormal_returns24, df_sanctions_24['TRADEDATE'], left_index=True, right_index=True)
abnormal_returns24


Beta for AFLT stock is = 1.14209 and alpha is = -0.0004
Beta for AFLT stock is = 1.15111 and alpha is = -0.00025
Beta for AFLT stock is = 1.44706 and alpha is = 0.00069
Beta for AFLT stock is = 0.25082 and alpha is = -0.00022
Beta for AFLT stock is = 0.94271 and alpha is = -0.00069
Beta for AFLT stock is = 0.77269 and alpha is = 0.0003
Beta for AFLT stock is = 0.28342 and alpha is = -0.00012
Beta for AFLT stock is = 0.96727 and alpha is = 0.00191
Beta for AFLT stock is = 0.79872 and alpha is = 0.00186
Beta for AFLT stock is = 0.31241 and alpha is = -0.00196
Beta for AFLT stock is = 0.88344 and alpha is = -0.00054
Beta for AFLT stock is = 1.0515 and alpha is = 0.00024
Beta for AFLT stock is = 1.45282 and alpha is = 1e-05
Beta for AFLT stock is = 0.0 and alpha is = 0.0
Beta for AFLT stock is = 0.08627 and alpha is = -0.00093
Beta for AFLT stock is = 0.83448 and alpha is = -0.00064
Beta for AFLT stock is = 1.21994 and alpha is = -0.00112
Beta for AFLT stock is = 1.27311 and alpha is = 0.0

,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,CARM,...,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX,TRADEDATE
724,-0.011685,0.008523,0.029062,-0.001438,0.015437,0.011748,0.020155,0.022802,0.030234,0.017686,...,0.000295,0.020934,0.016424,0.006925,0.010895,0.024247,0.001228,0.015976,0.004748,2024-01-03
725,0.003524,-0.006943,-0.008429,0.000706,-0.000786,0.024958,-0.002554,0.003155,-0.013626,0.011315,...,-0.004114,-0.001544,0.002794,0.004205,0.024911,0.086454,0.001773,0.001184,-0.006866,2024-01-04
726,-0.006004,0.035493,-0.007057,0.000408,0.004104,-0.005432,0.005158,0.003063,0.008161,0.010065,...,-0.003144,-0.000960,-0.000338,0.002985,-0.007262,-0.007477,0.038478,-0.005033,0.003660,2024-01-05
727,0.008698,0.009613,0.008240,-0.003033,-0.010309,0.005452,0.009040,0.004359,0.141412,-0.000245,...,-0.003158,0.010295,0.009212,0.022381,0.011497,0.000713,0.025454,-0.006185,0.005332,2024-01-08
728,-0.002916,0.006409,0.019621,-0.000070,0.013052,-0.003466,0.024604,-0.006853,-0.028081,0.017285,...,-0.000434,-0.006204,0.019492,0.042524,0.061392,0.078514,-0.004014,0.005017,-0.000067,2024-01-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
814,0.002350,0.022283,-0.024003,0.005966,0.014011,-0.004151,0.042292,0.002379,0.011187,0.008773,...,0.001627,-0.022861,-0.010094,-0.001474,0.003090,0.003584,-0.001426,-0.002745,0.009477,2024-05-13
815,0.033075,0.037121,-0.034026,-0.008337,-0.005576,-0.004594,-0.009796,-0.005582,-0.001329,0.004107,...,0.014697,0.007101,-0.009048,-0.011522,0.042442,-0.000374,-0.003803,0.057166,0.016673,2024-05-14
816,0.032601,0.044155,0.008940,-0.001939,0.004758,-0.001813,-0.005420,-0.008824,0.017185,0.022764,...,-0.004885,0.007444,-0.018786,-0.007748,0.019411,0.000646,0.004217,-0.027408,-0.032364,2024-05-15
817,0.007575,-0.019626,-0.017486,-0.007601,0.010433,-0.002643,-0.002631,-0.011425,0.013611,-0.009666,...,-0.000059,-0.005562,0.002493,-0.006067,-0.010095,-0.001929,-0.006389,-0.009453,-0.003467,2024-05-16


### Check the significance of coeff

2021

In [95]:
# df_feb23 = df_feb23.drop(columns='TRADEDATE')
res = []
for ticker in df_feb23.columns:
    X = add_constant(df_feb23['IMOEX'])
    y = df_feb23[ticker]
    model = OLS(y, X).fit()

    params = model.params
    p_values = model.pvalues
    
    # Проверяем значимость коэффициентов
    alpha_significant = p_values['const'] < 0.05
    beta_significant = p_values['IMOEX'] < 0.05
    
    # Добавляем результаты в список
    res.append({'TICKER': ticker,
                    'alpha_significant': alpha_significant,
                    'beta_significant': beta_significant})
    print(model.summary())

# Создаем DataFrame из списка результатов
df_results = pd.DataFrame(res)

# Выводим результаты
print(df_results)

                            OLS Regression Results                            
Dep. Variable:                   AFKS   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.365
Method:                 Least Squares   F-statistic:                     150.7
Date:                 Вс, 19 мая 2024   Prob (F-statistic):           1.24e-27
Time:                        22:53:49   Log-Likelihood:                 733.15
No. Observations:                 262   AIC:                            -1462.
Df Residuals:                     260   BIC:                            -1455.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0020      0.001     -2.228      0.0

/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountered in scalar divide
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_ts

                            OLS Regression Results                            
Dep. Variable:                   MRKU   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.059
Method:                 Least Squares   F-statistic:                     17.44
Date:                 Вс, 19 мая 2024   Prob (F-statistic):           4.06e-05
Time:                        22:53:49   Log-Likelihood:                 615.95
No. Observations:                 262   AIC:                            -1228.
Df Residuals:                     260   BIC:                            -1221.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0003      0.001      0.227      0.8

/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_tss
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1871: RuntimeWarning: invalid value encountered in scalar divide
  return self.mse_model/self.mse_resid
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:957: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/stats/stattools.py:50: RuntimeWarning: invalid value encountered in scalar divide
  dw = np.sum(diff_resids**2, axis=axis) / np.sum(resids**2, axis=axis)
/home/alexey/.local/lib/python3.10/site-packages/statsmodels/regression/linear_model.py:1782: RuntimeWarning: invalid value encountered in scalar divide
  return 1 - self.ssr/self.centered_ts

In [99]:
# df_feb23 = df_feb23.drop(columns='TRADEDATE')
res = []
for ticker in df_feb23.columns:
    X = add_constant(df_feb23['IMOEX'])
    y = df_feb23[ticker]
    model = OLS(y, X).fit()

    params = model.params
    p_values = model.pvalues
    
    # Определяем уровень значимости для каждого коэффициента
    if p_values['const'] < 0.001:
        alpha_significance = '0.1%'
    elif p_values['const'] < 0.01:
        alpha_significance = '1%'
    elif p_values['const'] < 0.05:
        alpha_significance = '5%'
    else:
        alpha_significance = 'Not significant'
        
    if p_values['IMOEX'] < 0.001:
        beta_significance = '0.1%'
    elif p_values['IMOEX'] < 0.01:
        beta_significance = '1%'
    elif p_values['IMOEX'] < 0.05:
        beta_significance = '5%'
    else:
        beta_significance = 'Not significant'
    
    # Добавляем результаты в список
    res.append({'TICKER': ticker,
                'alpha_significance': alpha_significance,
                'beta_significance': beta_significance})
    print(model.summary())

# Создаем DataFrame из списка результатов
df_results = pd.DataFrame(res)

# Выводим результаты
print(df_results)


                            OLS Regression Results                            
Dep. Variable:                   AFKS   R-squared:                       0.367
Model:                            OLS   Adj. R-squared:                  0.365
Method:                 Least Squares   F-statistic:                     150.7
Date:                 Вс, 19 мая 2024   Prob (F-statistic):           1.24e-27
Time:                        23:15:53   Log-Likelihood:                 733.15
No. Observations:                 262   AIC:                            -1462.
Df Residuals:                     260   BIC:                            -1455.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0020      0.001     -2.228      0.0

In [100]:
df_results

,TICKER,alpha_significance,beta_significance
0,AFKS,5%,0.1%
1,AFLT,Not significant,0.1%
2,AGRO,Not significant,0.1%
3,AKRN,5%,0.1%
4,ALRS,Not significant,0.1%
...,...,...,...
80,VKCO,5%,0.1%
81,VSMO,1%,5%
82,VTBR,Not significant,0.1%
83,WUSH,Not significant,Not significant


Make weights for overlap evenets

In [80]:
df_events = pd.read_csv('Event list for python.csv')
df_events

,NUMBER OF EVENT,EVENT DATE,BEGINNING DATE,END DATE
0,0,2022-03-24,2022-02-11,2022-04-07
1,1,2022-04-06,2022-03-27,2022-04-16
2,2,2022-04-07,2022-03-28,2022-04-17
3,3,2022-04-08,2022-03-29,2022-04-18
4,4,2022-06-03,2022-05-24,2022-06-13
5,5,2022-06-28,2022-06-18,2022-07-08
6,6,2022-07-21,2022-07-11,2022-07-31
7,7,2022-12-16,2022-12-06,2022-12-26
8,8,2023-02-24,2023-02-14,2023-03-06
9,9,2023-02-27,2023-02-15,2023-03-07


In [22]:
df_events['EVENT DATE'] = pd.to_datetime(df_events['EVENT DATE'])
df_events['BEGINNING DATE'] = pd.to_datetime(df_events['BEGINNING DATE'])
df_events['END DATE'] = pd.to_datetime(df_events['END DATE'])

# Функция для определения перекрывающихся событий
def find_overlapping_events(df_events):
    overlap_dict = {}
    for i, row in df_events.iterrows():
        event_number = row['NUMBER OF EVENT']
        start_date = row['BEGINNING DATE']
        end_date = row['END DATE']
        overlapping_events = df_events[(df_events['BEGINNING DATE'] <= end_date) & (df_events['END DATE'] >= start_date) & (df_events['NUMBER OF EVENT'] != event_number)]
        overlap_dict[event_number] = overlapping_events['NUMBER OF EVENT'].tolist()
    return overlap_dict

overlapping_events = find_overlapping_events(df_events)
print(overlapping_events)

{1: [2, 3, 4, 5, 6], 2: [1, 3, 4, 5, 6], 3: [1, 2, 4, 5, 6], 4: [1, 2, 3, 5, 6], 5: [1, 2, 3, 4, 6], 6: [1, 2, 3, 4, 5], 7: [8, 9], 8: [7, 9], 9: [7, 8], 10: [], 11: [], 12: [], 13: [], 14: [15], 15: [14], 16: [], 17: [], 18: [], 19: [20, 21], 20: [19, 21, 22], 21: [19, 20, 22], 22: [20, 21], 23: [], 24: []}


In [23]:
# Функция для расчета весов для каждого дня
def calculate_weights(df_events, overlapping_events):
    weights = {}
    for i, row in df_events.iterrows():
        event_number = row['NUMBER OF EVENT']
        start_date = row['BEGINNING DATE']
        end_date = row['END DATE']
        weights[event_number] = {}
        
        for single_date in pd.date_range(start_date, end_date):
            overlapping_days = sum([max(0, 21 - abs((single_date - df_events.loc[df_events['NUMBER OF EVENT'] == oe, 'EVENT DATE'].values[0]).days)) 
                                    for oe in overlapping_events[event_number]])
            weight = 1 / (1 + overlapping_days)
            weights[event_number][single_date] = weight
    
    return weights

weights = calculate_weights(df_events, overlapping_events)
print(weights)

{1: {Timestamp('2022-02-13 00:00:00'): 0.047619047619047616, Timestamp('2022-02-14 00:00:00'): 0.041666666666666664, Timestamp('2022-02-15 00:00:00'): 0.037037037037037035, Timestamp('2022-02-16 00:00:00'): 0.03333333333333333, Timestamp('2022-02-17 00:00:00'): 0.030303030303030304, Timestamp('2022-02-18 00:00:00'): 0.027777777777777776, Timestamp('2022-02-19 00:00:00'): 0.025, Timestamp('2022-02-20 00:00:00'): 0.022727272727272728, Timestamp('2022-02-21 00:00:00'): 0.020833333333333332, Timestamp('2022-02-22 00:00:00'): 0.019230769230769232, Timestamp('2022-02-23 00:00:00'): 0.017543859649122806, Timestamp('2022-02-24 00:00:00'): 0.016129032258064516, Timestamp('2022-02-25 00:00:00'): 0.015384615384615385, Timestamp('2022-02-26 00:00:00'): 0.014705882352941176, Timestamp('2022-02-27 00:00:00'): 0.014084507042253521, Timestamp('2022-02-28 00:00:00'): 0.013513513513513514, Timestamp('2022-03-01 00:00:00'): 0.013333333333333334, Timestamp('2022-03-02 00:00:00'): 0.013157894736842105, Tim

In [30]:
abnormal_returns_all = pd.concat([abnormal_returns22, abnormal_returns23, abnormal_returns24])
# abnormal_returns22_test = abnormal_returns22.set_index('TRADEDATE')
# abnormal_returns22_test
abnormal_returns_all = abnormal_returns_all.sort_values(by=['TRADEDATE'], ascending = True)
abnormal_returns_all = abnormal_returns_all.set_index('TRADEDATE')


In [59]:

# Пример данных об аномальных доходностях (AR)
# ar_data = {
#     'DATE': pd.date_range(start='2022-02-13', periods=500),  # Пример дат
#     'AR': np.random.randn(500)  # Пример случайных аномальных доходностей
# }
# ar_df = pd.DataFrame(ar_data)
# ar_df.set_index('DATE', inplace=True)

# Функция для расчета взвешенных AR
def calculate_weighted_ar(abnormal_returns_all, weights):
    weighted_ar = {}
    for event, dates in weights.items():
        weighted_ar[event] = {}
        for date, weight in dates.items():
            if date in abnormal_returns_all.index:
                for AR in abnormal_returns_all.columns:
                    weighted_ar[event][date] = abnormal_returns_all.loc[date, AR] * weight
    return weighted_ar

weighted_ar = calculate_weighted_ar(abnormal_returns_all, weights)

# # Функция для расчета CAR и CAAR
# def calculate_car(weighted_ar):
#     car = {event: sum(daily_ar.values()) for event, daily_ar in weighted_ar.items()}
    
#     return car

# car = calculate_car(weighted_ar)
# print(f"CAR: {car}")

weighted_ar

{1: {Timestamp('2022-02-14 00:00:00'): 0.0011400961007983087,
  Timestamp('2022-02-15 00:00:00'): 0.003920234589715956,
  Timestamp('2022-02-16 00:00:00'): 0.0004851228418606877,
  Timestamp('2022-02-17 00:00:00'): -0.0012062367321426146,
  Timestamp('2022-02-18 00:00:00'): -0.001527539256561112,
  Timestamp('2022-02-21 00:00:00'): -0.0008381810951098089,
  Timestamp('2022-02-22 00:00:00'): 0.00034740485358854934,
  Timestamp('2022-02-24 00:00:00'): -0.002477137867154222,
  Timestamp('2022-02-25 00:00:00'): -0.0025659269219968067},
 2: {Timestamp('2022-02-14 00:00:00'): 0.0010944922567663765,
  Timestamp('2022-02-15 00:00:00'): 0.0037802262115118144,
  Timestamp('2022-02-16 00:00:00'): 0.0004694737179296978,
  Timestamp('2022-02-17 00:00:00'): -0.0011707591811972436,
  Timestamp('2022-02-18 00:00:00'): -0.0014862544117891902,
  Timestamp('2022-02-21 00:00:00'): -0.0008210753584749149,
  Timestamp('2022-02-22 00:00:00'): 0.00034085004503027483,
  Timestamp('2022-02-24 00:00:00'): -0.002

In [60]:
def calculate_weighted_ar(abnormal_returns_all, weights):
    weighted_ar = {company: {} for company in abnormal_returns_all.columns}
    
    for event, dates in weights.items():
        for date, weight in dates.items():
            if date in abnormal_returns_all.index:
                for company in abnormal_returns_all.columns:
                    if company not in weighted_ar:
                        weighted_ar[company] = {}
                    if date not in weighted_ar[company]:
                        weighted_ar[company][date] = []
                    weighted_ar[company][date].append(abnormal_returns_all.loc[date, company] * weight)
    
    for company in weighted_ar:
        for date in weighted_ar[company]:
            weighted_ar[company][date] = np.sum(weighted_ar[company][date])
    
    return weighted_ar

weighted_ar = calculate_weighted_ar(abnormal_returns_all, weights)

# Функция для расчета AAR и CAAR
def calculate_aar_and_caar(weighted_ar):
    aar = pd.DataFrame(index=pd.to_datetime([]))
    for company in weighted_ar:
        temp_aar = {}
        for date in sorted(weighted_ar[company]):
            temp_aar[date] = weighted_ar[company][date]
        temp_aar = pd.Series(temp_aar)
        temp_aar.index = pd.to_datetime(temp_aar.index)
        aar[company] = temp_aar.groupby(temp_aar.index).mean()
    
    aar = aar.reindex(abnormal_returns_all.index).fillna(0)
    caar = aar.cumsum()
    return aar, caar

aar, caar = calculate_aar_and_caar(weighted_ar)
print(f"AAR:\n{aar}")
print(f"CAAR:\n{caar}")

AAR:
                AFKS      AFLT      AGRO      AKRN      ALRS      AQUA  ASTR  \
TRADEDATE                                                                      
2022-02-09  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   0.0   
2022-02-10  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   0.0   
2022-02-11  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   0.0   
2022-02-14  0.000928 -0.000373  0.000094 -0.002067 -0.000492 -0.000435   0.0   
2022-02-15  0.001806  0.001157 -0.000358  0.001462 -0.001794 -0.001642   0.0   
...              ...       ...       ...       ...       ...       ...   ...   
2024-05-13  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   0.0   
2024-05-14  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   0.0   
2024-05-15  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   0.0   
2024-05-16  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000   0.0   
2024-05-17  0.000000  0.000000  0.0

In [57]:
weighted_ar

{'AFKS': {Timestamp('2022-02-14 00:00:00'): 0.0009280976591691176,
  Timestamp('2022-02-15 00:00:00'): 0.0018057770935107336,
  Timestamp('2022-02-16 00:00:00'): 0.0027690970959824814,
  Timestamp('2022-02-17 00:00:00'): -0.0011118265395923203,
  Timestamp('2022-02-18 00:00:00'): -0.0026919955439847587,
  Timestamp('2022-02-21 00:00:00'): -0.0052946158697666935,
  Timestamp('2022-02-22 00:00:00'): 0.0069745757519881005,
  Timestamp('2022-02-24 00:00:00'): 0.0015611796160528507,
  Timestamp('2022-02-25 00:00:00'): -0.009933632805616682,
  Timestamp('2022-03-24 00:00:00'): -0.0017709944848335393,
  Timestamp('2022-03-25 00:00:00'): 0.0016176140842961858,
  Timestamp('2022-03-28 00:00:00'): -0.0050783180860117655,
  Timestamp('2022-03-29 00:00:00'): 9.362481527328117e-05,
  Timestamp('2022-03-30 00:00:00'): 0.004879065121791228,
  Timestamp('2022-03-31 00:00:00'): 0.0044902165314611555,
  Timestamp('2022-04-01 00:00:00'): -0.0017182408880910811,
  Timestamp('2022-04-04 00:00:00'): 0.00067

## AAR

In [95]:
abnormal_returns_all = abnormal_returns_all.drop(columns='IMOEX')

df_sum = abnormal_returns_all.sum(axis = 1)
df_count= abnormal_returns_all.count(axis = 1)
df_aar = df_sum / df_count
df_aar = pd.DataFrame(df_aar)

df_aar = df_aar.rename(columns={ 0 : 'AAR'})
df_aar

,AAR
TRADEDATE,
2022-02-09,0.006283
2022-02-10,-0.002059
2022-02-11,-0.010509
2022-02-14,0.010408
2022-02-15,0.002078
...,...
2024-05-13,0.004282
2024-05-14,-0.001513
2024-05-15,-0.003090


# CAR

In [81]:
df_events = df_events[['NUMBER OF EVENT', 'EVENT DATE']]
df_events

,NUMBER OF EVENT,EVENT DATE
0,0,2022-03-24
1,1,2022-04-06
2,2,2022-04-07
3,3,2022-04-08
4,4,2022-06-03
5,5,2022-06-28
6,6,2022-07-21
7,7,2022-12-16
8,8,2023-02-24
9,9,2023-02-27


In [82]:
results = []
for index, row in df_events.iterrows():
    event_date = row['EVENT DATE']
    event_number = row['NUMBER OF EVENT']
    
    event_date_num = pd.Timestamp(event_date).value

    start_index = abnormal_returns_all.index.get_loc(event_date) - 10
    end_index = abnormal_returns_all.index.get_loc(event_date) + 10 

    window_prices = abnormal_returns_all.iloc[start_index:end_index+1]

    company_sums = window_prices.sum()

    result_row = {'NUMBER OF EVENT': event_number}
    result_row.update(company_sums)
    results.append(result_row)

    
results


[{'NUMBER OF EVENT': 0,
  'AFKS': -0.06127113007460064,
  'AFLT': -0.23627411961694203,
  'AGRO': 0.10470881625306666,
  'AKRN': 0.37924152447692944,
  'ALRS': -0.03148066055642027,
  'AQUA': 0.08497917813748826,
  'ASTR': 0.0,
  'BANE': 0.04423392412213556,
  'BSPB': -0.08466323497095884,
  'CARM': 0.0,
  'CBOM': 0.0298630441620705,
  'CHMF': -0.24126001687104595,
  'CIAN': -0.24823464635355127,
  'DIAS': 0.0,
  'DSKY': -0.08987785876186005,
  'ELFV': 0.0,
  'ENPG': 0.045711063635334226,
  'ETLN': -0.12108307943399219,
  'FEES': -0.07411630855850482,
  'FIVE': -0.10735775360859559,
  'FIXP': -0.035646682963914406,
  'FLOT': -0.16549153206618963,
  'GAZP': 0.03409390345581457,
  'GCHE': 0.24769054269050286,
  'GEMC': -0.21846294196310262,
  'GLTR': -0.14430606238523686,
  'GMKN': 0.18549222455583825,
  'HHRU': -0.4578272470607543,
  'HNFG': 0.0,
  'HYDR': 0.16965462742089404,
  'IRAO': -0.09874180242569977,
  'LENT': 0.09047668483754641,
  'LKOH': -0.02277686322123925,
  'LSNG': 0.1615

In [83]:
results = pd.DataFrame(results)
results

,NUMBER OF EVENT,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,...,TRMK,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX
0,0,-0.061271,-0.236274,0.104709,0.379242,-0.031481,0.084979,0.000000,0.044234,-0.084663,...,0.042267,-0.202933,0.000000,-0.299010,-0.191187,-0.005867,0.067086,-0.295204,0.000000,-0.306102
1,1,-0.088595,-0.153459,0.007251,0.226395,-0.082607,0.053642,0.000000,-0.068372,-0.010398,...,0.009370,-0.016824,0.000000,-0.134347,-0.219937,0.316646,0.217324,-0.168488,0.000000,-0.173400
2,2,0.111917,-0.092975,0.101818,0.235902,0.061500,0.044728,0.000000,0.005095,-0.151147,...,0.046796,-0.171088,0.000000,-0.128524,-0.105977,0.440156,0.141707,0.017958,0.000000,-0.069490
3,3,0.125226,0.108601,0.088413,0.252186,-0.005572,0.031707,0.000000,0.047030,-0.100508,...,0.101426,-0.079882,0.000000,-0.116905,-0.093185,0.451186,0.221975,0.124295,0.000000,0.003464
4,4,0.015021,-0.021250,-0.114870,-0.091942,-0.101460,0.101644,0.000000,0.154741,0.038523,...,-0.115552,0.152049,0.000000,0.065135,-0.397589,-0.113613,-0.022462,-0.016024,0.000000,-0.051649
5,5,0.234029,0.100525,0.021512,0.193638,0.001747,0.012027,0.000000,-0.053249,0.055022,...,0.336564,-0.079974,0.000000,0.149259,-0.053277,0.202288,-0.095610,0.126988,0.000000,0.248746
6,6,0.022494,0.023708,-0.024692,-0.160198,-0.018965,0.006641,0.000000,-0.093635,0.207260,...,0.237543,-0.146007,0.000000,0.007971,0.005013,0.312767,-0.044864,-0.001229,0.000000,0.252722
7,7,0.003672,0.020341,-0.140972,-0.051951,-0.092461,-0.109745,0.000000,0.029254,0.007166,...,0.055417,-0.014057,0.000000,-0.015461,-0.080376,-0.096873,-0.066495,-0.024108,-0.134919,-0.047348
8,8,0.010999,-0.014673,0.017076,-0.088672,0.010032,-0.015695,0.000000,0.046212,0.039402,...,-0.045361,0.017005,0.000000,0.108617,0.166717,0.019947,-0.061414,-0.001315,0.148344,-0.053867
9,9,0.004611,-0.011191,0.019834,-0.078334,0.015216,-0.022506,0.000000,0.051537,-0.030589,...,-0.026416,0.021632,0.000000,0.104652,0.085915,0.045368,-0.074701,0.000965,0.169310,-0.052051


# CAAR

In [99]:
results = []
for index, row in df_events.iterrows():
    event_date = row['EVENT DATE']
    event_number = row['NUMBER OF EVENT']
    
    event_date_num = pd.Timestamp(event_date).value

    start_index = df_aar.index.get_loc(event_date) - 10
    end_index = df_aar.index.get_loc(event_date) + 10 

    window_prices = df_aar.iloc[start_index:end_index+1]

    company_sums = window_prices.sum()

    result_row = {'NUMBER OF EVENT': event_number}
    result_row.update(company_sums)
    results.append(result_row)

    
results = pd.DataFrame(results)
results = results.rename(columns={'AAR' : 'CAAR'})
results

,NUMBER OF EVENT,CAAR
0,0,-0.052727
1,1,0.006096
2,2,0.059231
3,3,0.063549
4,4,-0.050266
5,5,0.028235
6,6,0.012627
7,7,-0.006164
8,8,0.011137
9,9,0.011960


# Linear regression

In [ ]:
# Data processing
df = pd.read_csv('./stock_price.csv')
df = df [['TRADEDATE','CLOSE', 'TICKER']]
df2 = pd.read_excel('IMOEX.xlsx')

# df2['TRADEDATE'] = df2['TRADEDATE'].astype(str).str.replace('.', '-')
# # df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])
df2['TRADEDATE'] = pd.to_datetime(df2['TRADEDATE'],format='%d.%m.%Y' )
df= pd.concat([df, df2])
df['CLOSE'] = pd.to_numeric(df['CLOSE'], errors='coerce')
# df['VOLUME'] = pd.to_numeric(df['VOLUME'], errors='coerce')
# df['VOLUME'] = df['VOLUME'].fillna(0)
# df['CLOSE'] = df['CLOSE'].fillna()



df = df.loc[df['TRADEDATE'] != 'TRADEDATE']
df = df.dropna()
df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])


df = df.sort_values(by='TRADEDATE', ascending=True)

df

In [114]:
df_cap = pd.read_csv('stock_prices_market_cap.csv')
df_cap = df_cap.loc[df_cap['TRADEDATE']!= 'TRADEDATE']
df_cap = df_cap[['TRADEDATE', 'TICKER', 'SIZE']]
df_cap['TRADEDATE'] = pd.to_datetime(df_cap['TRADEDATE'])
# df_cap = df_cap.drop_duplicates(subset=['TRADEDATE'])
df_cap = df_cap.dropna()
df_cap['TRADEDATE'] = pd.to_datetime(df_cap['TRADEDATE'])
df_cap = df_cap.pivot(index = 'TRADEDATE', columns='TICKER', values = 'SIZE' )
df_cap = df_cap.fillna(0)
df_cap


TICKER,AFKS,AFLT,AGRO,AKRN,ALRS,AQUA,ASTR,BANE,BSPB,CARM,...,TRMK,TRNFP,UGLD,UPRO,VEON-RX,VKCO,VSMO,VTBR,WUSH,YNDX
TRADEDATE,,,,,,,,,,,,,,,,,,,,,
2021-02-01,1.834146e+09,7.653358e+08,2.282730e+08,19853050.0,1.590618e+09,6949590.0,0.000000e+00,23001007.5,12311834.0,0.000000e+00,...,42228123.0,1.092902e+08,0.000000e+00,1.932555e+08,0.0,0.000000e+00,15834060.0,7.713843e+08,0.000000e+00,2.784128e+09
2021-02-02,3.810827e+09,9.378398e+08,1.374396e+08,11618976.0,2.549610e+09,7619315.0,0.000000e+00,10610720.0,11388883.9,0.000000e+00,...,27388746.0,1.239978e+08,0.000000e+00,1.604354e+08,0.0,0.000000e+00,8728000.0,1.032231e+09,0.000000e+00,4.231724e+09
2021-02-03,2.064138e+09,8.581874e+08,1.633517e+08,14846166.0,2.011171e+09,8020290.0,0.000000e+00,12972579.5,10218752.8,0.000000e+00,...,28405977.6,9.726750e+07,0.000000e+00,1.517813e+08,0.0,0.000000e+00,6076620.0,8.367657e+08,0.000000e+00,3.316538e+09
2021-02-04,1.441797e+09,6.739503e+08,1.155906e+08,11888924.0,1.260884e+09,4237500.0,0.000000e+00,8079710.0,31827978.0,0.000000e+00,...,10996136.0,1.965674e+08,0.000000e+00,1.338719e+08,0.0,0.000000e+00,88909100.0,6.889123e+08,0.000000e+00,3.445326e+09
2021-02-05,2.333466e+09,7.901522e+08,7.530580e+07,17118990.0,1.322600e+09,2168640.0,0.000000e+00,12795867.0,15288728.5,0.000000e+00,...,6783882.6,1.857142e+08,0.000000e+00,1.283960e+08,0.0,0.000000e+00,9190680.0,1.128693e+09,0.000000e+00,2.920015e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-13,1.835001e+09,1.170801e+09,1.344996e+09,44287640.0,9.929276e+08,67144506.0,2.018806e+09,104172630.0,458264790.0,9.483075e+07,...,337601549.4,1.382665e+09,1.179876e+09,6.265421e+08,249245.0,4.333993e+08,30351360.0,1.585397e+09,1.269181e+08,2.033877e+10
2024-05-14,2.448678e+09,3.253330e+09,6.557376e+08,74104800.0,8.090144e+08,65705425.0,9.535590e+08,61110000.0,456418560.0,2.385851e+07,...,251381975.0,3.808302e+09,4.947358e+08,4.091963e+08,3082253.0,3.289851e+09,27582540.0,1.112811e+09,9.784723e+08,6.895807e+09
2024-05-15,4.190175e+09,4.918794e+09,3.590842e+08,48907152.0,8.587524e+08,65275648.0,4.575397e+08,70079988.0,634449172.3,9.093285e+07,...,188888503.4,1.526344e+09,7.828912e+08,3.707509e+08,1367908.5,2.791777e+09,23427360.0,1.485876e+09,1.291975e+09,5.399128e+09
